# TF Tutorial

In [2]:
import os
import tensorflow as tf
import numpy as np


In [3]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [4]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

In [5]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size, architecture = 'LSTM', layers = 1):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                batch_input_shape=[batch_size,None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences = True, stateful = True,
                recurrent_initializer = 'glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [6]:
def train_model(name, EPOCHS = 100):
    if name == 'generic':
        text = open('./texts/essays.txt', 'rb').read().decode(encoding='utf-8')
        
    else:

        text = open('./texts/{}.txt'.format(name), 'rb').read().decode(encoding='utf-8')
        essays = open('./texts/essays.txt', 'rb').read().decode(encoding='utf-8')

        text = text + essays
    print(len(text))
    vocab = sorted(set(text))
    
    char2idx = {unique:idx for idx, unique in enumerate(vocab)}
    idx2char = np.array(vocab)
    text_as_int = np.array([char2idx[char] for char in text])
    seq_len = 100

    examples_per_epoch = len(text) // (seq_len + 1)
    char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
    sequences = char_dataset.batch(seq_len + 1, drop_remainder = True)
    dataset = sequences.map(split_input_target)
    BATCH_SIZE = 64
    BUFFER_SIZE = 10000

    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

    vocab_size = len(vocab)
    
    embedding_dim = 256
    rnn_units = 1024
    
    model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = BATCH_SIZE)
    
    model.compile(optimizer = 'adam', loss = loss)
    checkpoint_path = './final_weights/{}/'.format(name)
    try:
        model.load_weights(checkpoint_path)
        print('model loaded')
    except:
        pass
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True)
    
    history = model.fit(dataset, epochs = EPOCHS, callbacks = [cp_callback])
    
    return history

In [20]:
text = open('./texts/marx.txt', 'rb').read().decode(encoding='utf-8')
essays = open('./texts/essays.txt', 'rb').read().decode(encoding='utf-8')

text = text + essays

In [7]:
print(len(text))

573715


In [44]:
vocab = sorted(set(text))

In [45]:
char2idx = {unique:idx for idx, unique in enumerate(vocab)}
idx2char = np.array(vocab)

In [46]:
text_as_int = np.array([char2idx[char] for char in text])

In [47]:
seq_len = 100

examples_per_epoch = len(text) // (seq_len + 1)

In [48]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [49]:
sequences = char_dataset.batch(seq_len + 1, drop_remainder = True)


In [50]:
dataset = sequences.map(split_input_target)

In [51]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

vocab_size = len(vocab)

In [52]:
embedding_dim = 256
rnn_units = 1024

In [53]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = BATCH_SIZE)

In [54]:
model.compile(optimizer = 'adam', loss = loss)

In [55]:
len(text)

573715

In [56]:
#checkpoint_dir = './training_checkpoints'
#checkpoint_prefix = os.path.join(checkpoint_dir, 'marx_{epoch}')
#checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#    filepath = checkpoint_prefix, save_weights_only = True)
checkpoint_path = './final_weights/marx/'
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True)

In [57]:
EPOCHS = 100

In [58]:
history = model.fit(dataset, epochs = EPOCHS, callbacks = [cp_callback])

Train for 88 steps
Epoch 1/100
88/88 [==============================] - 5s 58ms/step - loss: 2.9391
Epoch 2/100
63/88 [====================>.........] - ETA: 1s - loss: 2.4375

KeyboardInterrupt: 

In [25]:
generative_model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = 1)

In [26]:
generative_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [27]:
generative_model.build(tf.TensorShape([1,None]))

In [38]:
def gen_text(model, start_string):
    num_gen = 1000
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    temperature = 1.0
    model.reset_states()
    
    for i in range(num_gen):
        predictions = generative_model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1,0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    while idx2char[predicted_id] != '.':
        predictions = generative_model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1,0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    ret_str = start_string + ''.join(text_generated)
    ret_str = ret_str.replace('\n', ' ')
    ret_str = ret_str.split('.')
    ret_str = '.'.join(ret_str[1:])
    return (ret_str)

In [39]:
i = int(np.random.randint(0, len(text)-100))
start_text = text[i:int(i+100)]
generated_text = gen_text(generative_model, start_text)

In [40]:
generated_text


' These “Wages of Whiteness,” as Roediger described them to discover, expecial in the production of history’s direction, his abstraction is not a mutual misunderstanding of companishtof  of the divergent person and to the absolute monarchy and "Yak at ho wine  Within Ro  some of this tas  A  discrepancy of individuals; all  they makes as he is from his fellow, although it is manipulated by his practices and are struggle into three age groups will achieved though it mythologies from the collition of bourgeois property But in the bourgeois, as well as their own cultural signifiers to gradation of cyit is self-redamental right to the proletariat are made up of time Both are es helped on diamonds and  durkhe marooon” as a multiplicity and lies him enarche’s claims, with local corresponding of society more immediately begin Spelke and Thoe chayit is not require the same abort from the first monkey recognition to fight naked by protection of its cultural outcoinsion of capital and loces that

In [ ]:

for file in os.listdir('./texts'):
    print(file[:-4])
    history = train_model(str(file[:-4]))


freud
281359
model loaded
Train for 43 steps
Epoch 1/100
43/43 [==============================] - 3s 78ms/step - loss: 0.5609
Epoch 2/100
43/43 [==============================] - 2s 49ms/step - loss: 0.3589
Epoch 3/100
43/43 [==============================] - 2s 47ms/step - loss: 0.3269
Epoch 4/100
43/43 [==============================] - 2s 49ms/step - loss: 0.3138
Epoch 5/100
43/43 [==============================] - 2s 50ms/step - loss: 0.3114
Epoch 6/100
43/43 [==============================] - 2s 48ms/step - loss: 0.3050
Epoch 7/100
43/43 [==============================] - 2s 49ms/step - loss: 0.2999
Epoch 8/100
43/43 [==============================] - 2s 48ms/step - loss: 0.2959
Epoch 9/100
43/43 [==============================] - 2s 50ms/step - loss: 0.2932
Epoch 10/100
43/43 [==============================] - 2s 50ms/step - loss: 0.2910
Epoch 11/100
43/43 [==============================] - 2s 50ms/step - loss: 0.2885
Epoch 12/100
43/43 [==============================] - 3s 64ms/

43/43 [==============================] - 2s 51ms/step - loss: 0.2354
Epoch 43/100
43/43 [==============================] - 2s 50ms/step - loss: 0.2371
Epoch 44/100
43/43 [==============================] - 2s 48ms/step - loss: 0.2353
Epoch 45/100
43/43 [==============================] - 2s 49ms/step - loss: 0.2340
Epoch 46/100
43/43 [==============================] - 2s 50ms/step - loss: 0.2333
Epoch 47/100
43/43 [==============================] - 2s 52ms/step - loss: 0.2314
Epoch 48/100
43/43 [==============================] - 3s 62ms/step - loss: 0.2299
Epoch 49/100
43/43 [==============================] - 2s 50ms/step - loss: 0.2285
Epoch 50/100
43/43 [==============================] - 2s 47ms/step - loss: 0.2241
Epoch 51/100
43/43 [==============================] - 2s 48ms/step - loss: 0.2257
Epoch 52/100
43/43 [==============================] - 2s 47ms/step - loss: 0.2279
Epoch 53/100
43/43 [==============================] - 2s 48ms/step - loss: 0.2267
Epoch 54/100
43/43 [=========

109/109 [==============================] - 5s 47ms/step - loss: 0.2259
Epoch 41/100
109/109 [==============================] - 5s 46ms/step - loss: 0.2241
Epoch 42/100
109/109 [==============================] - 5s 45ms/step - loss: 0.2217
Epoch 43/100
109/109 [==============================] - 5s 47ms/step - loss: 0.2222
Epoch 44/100
109/109 [==============================] - 5s 45ms/step - loss: 0.2207
Epoch 45/100
109/109 [==============================] - 5s 46ms/step - loss: 0.2202
Epoch 46/100
109/109 [==============================] - 5s 47ms/step - loss: 0.2232
Epoch 47/100
109/109 [==============================] - 5s 48ms/step - loss: 0.2229
Epoch 48/100
109/109 [==============================] - 5s 44ms/step - loss: 0.2246
Epoch 49/100
109/109 [==============================] - 5s 46ms/step - loss: 0.2282
Epoch 50/100
109/109 [==============================] - 5s 47ms/step - loss: 0.2309
Epoch 51/100
109/109 [==============================] - 5s 49ms/step - loss: 0.2322
Epoch

84/84 [==============================] - 4s 47ms/step - loss: 0.2373
Epoch 39/100
84/84 [==============================] - 4s 45ms/step - loss: 0.2367
Epoch 40/100
84/84 [==============================] - 4s 47ms/step - loss: 0.2370
Epoch 41/100
84/84 [==============================] - 4s 45ms/step - loss: 0.2362
Epoch 42/100
84/84 [==============================] - 4s 46ms/step - loss: 0.2370
Epoch 43/100
84/84 [==============================] - 4s 46ms/step - loss: 0.2370
Epoch 44/100
84/84 [==============================] - 4s 47ms/step - loss: 0.2354
Epoch 45/100
84/84 [==============================] - 4s 48ms/step - loss: 0.2356
Epoch 46/100
84/84 [==============================] - 4s 45ms/step - loss: 0.2342
Epoch 47/100
84/84 [==============================] - 4s 45ms/step - loss: 0.2346
Epoch 48/100
84/84 [==============================] - 4s 48ms/step - loss: 0.2322
Epoch 49/100
84/84 [==============================] - 4s 46ms/step - loss: 0.2323
Epoch 50/100
84/84 [=========

96/96 [==============================] - 5s 47ms/step - loss: 0.2021
Epoch 38/100
96/96 [==============================] - 4s 47ms/step - loss: 0.2019
Epoch 39/100
96/96 [==============================] - 5s 48ms/step - loss: 0.2013
Epoch 40/100
96/96 [==============================] - 4s 46ms/step - loss: 0.1993
Epoch 41/100
96/96 [==============================] - 4s 45ms/step - loss: 0.2008
Epoch 42/100
96/96 [==============================] - 5s 48ms/step - loss: 0.2018
Epoch 43/100
96/96 [==============================] - 4s 45ms/step - loss: 0.2032
Epoch 44/100
96/96 [==============================] - 5s 49ms/step - loss: 0.2019
Epoch 45/100
96/96 [==============================] - 4s 46ms/step - loss: 0.2014
Epoch 46/100
96/96 [==============================] - 4s 44ms/step - loss: 0.2014
Epoch 47/100
96/96 [==============================] - 5s 47ms/step - loss: 0.1994
Epoch 48/100
96/96 [==============================] - 5s 49ms/step - loss: 0.1992
Epoch 49/100
96/96 [=========

88/88 [==============================] - 4s 48ms/step - loss: 0.4357
Epoch 37/100
88/88 [==============================] - 4s 46ms/step - loss: 0.5098
Epoch 38/100
88/88 [==============================] - 4s 47ms/step - loss: 0.4291
Epoch 39/100
88/88 [==============================] - 4s 47ms/step - loss: 0.3503
Epoch 40/100
88/88 [==============================] - 4s 47ms/step - loss: 0.2916
Epoch 41/100
88/88 [==============================] - 4s 46ms/step - loss: 0.2493
Epoch 42/100
88/88 [==============================] - 4s 47ms/step - loss: 0.2211
Epoch 43/100
88/88 [==============================] - 4s 46ms/step - loss: 0.2004
Epoch 44/100
88/88 [==============================] - 4s 47ms/step - loss: 0.1885
Epoch 45/100
88/88 [==============================] - 4s 50ms/step - loss: 0.1803
Epoch 46/100
88/88 [==============================] - 4s 50ms/step - loss: 0.1740
Epoch 47/100
88/88 [==============================] - 4s 48ms/step - loss: 0.1704
Epoch 48/100
88/88 [=========

In [11]:
history = train_model('locke')



409739
Train for 63 steps
Epoch 1/100
63/63 [==============================] - 4s 65ms/step - loss: 2.9613
Epoch 2/100
63/63 [==============================] - 3s 46ms/step - loss: 2.3745
Epoch 3/100
63/63 [==============================] - 3s 48ms/step - loss: 2.1205
Epoch 4/100
63/63 [==============================] - 3s 46ms/step - loss: 1.8965
Epoch 5/100
63/63 [==============================] - 3s 44ms/step - loss: 1.7215
Epoch 6/100
63/63 [==============================] - 3s 46ms/step - loss: 1.5646
Epoch 7/100
63/63 [==============================] - 3s 49ms/step - loss: 1.4589
Epoch 8/100
63/63 [==============================] - 3s 49ms/step - loss: 1.3816
Epoch 9/100
63/63 [==============================] - 3s 49ms/step - loss: 1.3213
Epoch 10/100
63/63 [==============================] - 3s 47ms/step - loss: 1.2725
Epoch 11/100
63/63 [==============================] - 3s 49ms/step - loss: 1.2304
Epoch 12/100
63/63 [==============================] - 3s 48ms/step - loss: 1.1954

63/63 [==============================] - 3s 52ms/step - loss: 0.2056


In [62]:
i = int(np.random.randint(0, len(text)-100))
start_text = text[i:int(i+100)]
generated_text = gen_text(generative_model, start_text)

In [63]:
generated_text

"  Durkheim, they, the peasant, a little do they had nothing to gain, and even the linen's own which subjects behaved either selfishled in drawing any absolute monarchy as a counterpois become more voluming the Devil, both bence the power of government, as they forgeour theoletical holding society scared of labour, so the landlord, the solid may considered useful  This property of altruism and staring and appropriating out we shall her cast, buth their philosophizing of this factions As Saci the language of colonialism In fact, we see very specifically the bourgeoisie itself, to the Deich that of the commodity considers to draws on comprehend its political power for the different kinds of commodities that the British position of the protective other in the relation of  commodity to commodity In fact, the struggle of the proletariat Racism, too, people spin and hoperty, the modern viewer reasoning information helps must focus a member or or  any ways in production More interested indivi

In [8]:
from that_text_gen import model as generate_model


In [34]:
model = generate_model('freud')

In [35]:
model.load_model()

In [36]:
test = model.generate_text(num = 1000)


In [37]:
test

' \tInstead, Hegel proposes that world history is dichat variod matters that have cause for love and displace it to the will of a particular  ego revilw, the peculimarily chose that learning dulater there was a perfectly grading the need for  away f the ego-ideal ans cathexis and identification. The trial attempt $ — and Sepoys providing somewhat of the individual, then, spheimers belafound them to self-interest inclusive concepts, and three distinct transmment of the civilization.  He argues that if [the case it is much no less new demonstrate on race of integains are a mass of reason, and thus are apart from the ego and which at the barmin developed by constance can and unification  with it for empathy has born furthed movements rulems of the motivation for the custom can only capitalism. For in his own do sools, ifertain ideological properties on the ideal in relation to his mechanism, he or she is instinct of the edhe Blobs Chinke wools for the control of the masses, both ones indi

In [12]:
for file in os.listdir('./texts'):
    print(file[:-4])
    model = generate_model(file[:-4])
    model.load_model()
    test = model.generate_text(num = 1000)
    print(test)


freud
 This leads us back to exert of their own. 	In order to understan a situation, a tester complained to rely similarities, every individual is considered to be examined through an individual acts of opposition undertaken by people who are unable or otherwise not disposed to pass misinghough the atrocitize the co increasing the amount of weak not the case of the Sepoy Rebellion as the physician  and the domi^  thin these systems, Fanon argues, have broadly cooperation helps the other player while reducing the reward for the cooperator. 	While defecting in security, wherefore the state of nature formulated, also require to be put    34     THE EGO AND THE SUPER-EGO (EGO-IDEAL) 35   gone into is the self-preservative institutions and proto   48t actual notion of integestate of nature as truly natural; nor ior of the other would be a mistake. An useld hid to find its work of mastering  the tensions.     V    THE SUBORDINATE RELATIONSHIPS  OF THE EGO   F This resistance must to give up 

” His sentence continues, however, by referring to the goal of “dispositions” thus actualizes the homosexual man is alleged theresting’s appropriation and refusals are themselves refused, a double negation that redoublesent and generative convergences from the people within philosophical person a subversive possibility to it, the body could shapped in other ways of arbitrary relation between such acts, for example, however, Kristeva suggests that, because the maternal body signifies the source from which women (reformulated altruism.  This impossible gender confined by the cultural possibility regardless of the cultural presence of AIDS, In rmand the complexity of these fields of powe (197). But what would consider the way in which that project is also fragion of the “before” within feminist theory becomes politically problematic in if thus first to be fundamentally phantasmatic stresed to overthrow those official structure, in pobithe state of nature.
marx
 The ure as a who in the res

  Hobbes proposes that for this reason, all of the power should be given to one occasie actively opposed to them. The stupid player is that this conception of the ruling class, but it is ceded to give a personal essence as  ‘mere appearance’; - but this reflexive sublation of the immediate, this  positing of its symbolic can thought itself- which was thus give up as the training of soldiers; it is necessary to the breakdomn Economy, he is still living, but as a synthetic cry (p < . . . He through should  fill in the le of capitalism to every corner of the world via colonialism is useful for capitalism, then, not just because it is constitu-  tively an ‘external’ play external: it is at the same  time supposed to confess - to determine himself, ature, at least formally, from power, they are most likely to redirect violence not exist.   ‘found her merchant of colonialism and imperialism served to reinforce their new position as a maximal coincidence, and ns for a simple answers, which br

  Empathy was measured both through explications of Which the plantation of the role of government is limited by the extent to which it is in the enlightened telping a state of nature that is various species being. The Latin American Inquisitors were much more concerned with keeping the newly founded Latin American elite in line than they were dealing with the minutia of the day to day sins of the poor. By cay a whole, happealing relationships between individuals due to the division of labor.  This cohesion, a trsaid for an action thut develops many do not all ontic systems. For egoistic man is permitted into the sphere of politics only insofar as he divorces himself in this role from the actual man, embedded in a real effect on the extrence of déjà vu also suggests a dual memory system.  Specifically, this experience is marked by a noted sense of familiarity that is then removed, followed by a buffer period over which the Blob must be able to remember which image itself in reach for i

  The systemic is evidence to other men. Con-  set of the world (27).  She fathes ]  and is corper-  the uses normal been considerable progressively  slacken.   This law was wrong, but  made more systems, whereas we have as just seen that could demonstrate critique. We shall respect as violence inwards against the Mundurukú and the particular familian control test, as happening their own lead in a multiple guilt by nocit that crimen intermed only by the greater or  l esser. In the world (27).  According to him, radical, must have no other  cause violence in two persons of the individual may make the same in point, abnot make judgment from the civil law delay is so generated by monkeys blassess them in  classes and which must  be impossible to cle of the point was essentially relieving the larter dicates by punishment for the sake of free thought that the Mundurukú alst since it is rejeence to contrary, the evolution to a social liffere, privileged theory from the majority as is made ea

 One should not wish to be the final eliging systems must be compared. 	Hegel, Leviathan 223) ideas of whose like a legicians became sone or greater abserving to their own wealth in the state of nature, and thus "it is not only could dom it is option—that any knowing that only fix thought of suicide is a greater continued variety of even at the heaven of OUR life. There are few pains so grievous as to have seen, divined, generally, one that is not obliged to straighten and should have some control over the points of kester in e the nobility of the world, and interpreting of the world in the more imp faccly as "should only begin to systems to maintain a stable community of cooperators, who MacGaffey characterizes as a “dialogue of the deaf.”  This easily need of a bell-wether of reason, a trine object permanence is dictated by a grand design, whose architect is regarded as morally able to avoid impense and blame have been able to conclude than any one should have acquired throughout the